In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

plt.style.use('seaborn-whitegrid')

from sklearn.preprocessing import MinMaxScaler

from keras.models import Model
from keras.layers import Input, Conv2D, MaxPool2D, Dense
from keras.layers import Flatten, Dropout
from keras.callbacks import EarlyStopping

import os
print(os.listdir("../input/flowerdataset/flowerDataset/"))

In [ ]:
train_df = pd.read_csv("../input/flowerdataset/flowerDataset/flowers17_training.csv", header=None)
test_df = pd.read_csv("../input/flowerdataset/flowerDataset/flowers17_testing.csv", header=None)

In [ ]:
train_df.head()

In [ ]:
y_train = train_df[0]
X_train = train_df.drop(columns=[0])
y_test = test_df[0]
X_test = test_df.drop(columns=[0])

In [ ]:
y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

In [ ]:
X_train.head()

In [ ]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [ ]:
X_train = X_train.reshape((X_train.shape[0], 64,64,3))
X_test = X_test.reshape((X_test.shape[0], 64, 64, 3))

In [ ]:
def show_images(images, cols = 1, titles = None):
    """Display a list of images in a single figure with matplotlib.
    
    Parameters
    ---------
    images: List of np.arrays compatible with plt.imshow.
    
    cols (Default = 1): Number of columns in figure (number of rows is 
                        set to np.ceil(n_images/float(cols))).
    
    titles: List of titles corresponding to each image. Must have
            the same length as titles.
    """
    assert((titles is None)or (len(images) == len(titles)))
    n_images = len(images)
    if titles is None: print('Serial title'); titles = ['Image (%d)' % i for i in range(1,n_images + 1)]
    fig = plt.figure()
    for n, (image, title) in enumerate(zip(images, titles)):
        a = fig.add_subplot(cols, np.ceil(n_images/float(cols)), n + 1)
        if image.ndim == 2:
            plt.gray()
        plt.imshow(image.reshape((64,64,3)), cmap=None)
        a.set_title(title, fontsize=50)
        a.grid(False)
        a.axis("off")
    fig.set_size_inches(np.array(fig.get_size_inches()) * n_images)
plt.show()

In [ ]:
samples = np.random.choice(len(X_train), 8)
sample_images = []
sample_labels = []
for sample in samples:
    sample_images.append(X_train[sample])
    sample_labels.append(np.argmax(y_train.iloc[sample]))

In [ ]:
show_images(sample_images, 2, titles=sample_labels)

In [ ]:
def modelCNN(inputShape, classes):    
    inputX = Input(inputShape)

    l1 = Conv2D(32, (3, 3), padding="same", activation='relu')(inputX)
    l1 = Conv2D(32, (3, 3), padding="same", activation='relu')(l1)
    l1 = MaxPool2D(pool_size=(2, 2))(l1)
    l1 = Dropout(0.25)(l1)

    # second CONV => RELU => CONV => RELU => POOL layer set
    l2 = Conv2D(64, (3, 3), padding="same", activation='relu')(l1)
    l2 = Conv2D(64, (3, 3), padding="same", activation='relu')(l2)
    l2 = MaxPool2D(pool_size=(2, 2))(l2)
    l2 = Dropout(0.25)(l2)

    # third CONV => RELU => CONV => RELU => POOL layer set
    l3 = Conv2D(128, (3, 3), padding="same", activation='relu')(l2)
    l3 = Conv2D(128, (3, 3), padding="same", activation='relu')(l3)
    l3 = MaxPool2D(pool_size=(2, 2))(l3)
    l3 = Dropout(0.25)(l3)

    # first (and only) set of FC => RELU layers
    l4 = Flatten()(l3)
    l4 = Dense(512, activation="relu")(l4)
    l4 = Dropout(0.5)(l4)

    # softmax classifier
    predictions = Dense(classes, activation="softmax")(l4)
    
    modelCNN = Model(inputs=inputX, outputs=predictions)
    
    return modelCNN

In [ ]:
model = modelCNN((64,64,3), 17)

In [ ]:
model.summary()

In [ ]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=['accuracy'])

In [ ]:
earlyStopper = EarlyStopping(monitor='acc', patience=1, restore_best_weights=True)

In [ ]:
hist = model.fit(x=X_train, y=y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), callbacks=[earlyStopper])

In [ ]:
# plot the training loss and accuracy
plt.figure()
plt.plot(hist.history["loss"], label="train_loss")
plt.plot(hist.history["val_loss"], label="val_loss")
plt.plot(hist.history["acc"], label="train_acc")
plt.plot(hist.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="center right");